In [ ]:
import csv
from dotenv import load_dotenv
import os
import requests

In [ ]:
load_dotenv()

URL_UPW = os.getenv("URL_UPW")

In [ ]:
with open("output/doi_of_publications_from_openalex_missing_in_fosm_from_2021.csv", newline='') as csvfile:
  rows = csv.reader(csvfile)
  dois = [row[0] for row in rows]
  list = []
  for doi in dois:
    url = f"http://doi.org/{doi}"
    unpaywall_url = f"https://api.unpaywall.org/v2/{doi}?email=bso@recherche.gouv.fr"
    unpaywall_response = requests.get(unpaywall_url)
    if unpaywall_response.status_code != 200:
      print(f"error while fetching data from unpaywall for DOI {doi}")
      continue
    title = unpaywall_response.json().get("title")
    if not title:
      print(f"unpaywall has no title for DOI {doi}")
      continue
    if len(title) == 0:
      print(f"title is empty string on unpaywall for DOI {doi}")
      continue
    list.append({ "url": url, "title": title })

requests.post(f"{URL_UPW}/forward", json={ "url": "http://crawler:5001/crawl", "params": {"list": list } })